# Process indexes

This notebook calculates key Economic Complexity indicators: RCA, M matrix, ECI, PCI, and opportunity metrics such as proximity, density, and strategic value. These are used to evaluate the productive structure of states and identify diversification paths.

In [1]:
import pandas as pd
import numpy as np

## 1. Read data

Read the necessary files and select which data and aggregation you want to use

In [ ]:
# Select which data and aggregation to use

data_choice = "trade" # trade / labor
labor_aggregation = "state" # state / metropolitan_area

In [100]:
# Directory structure and read database

if data_choice == "trade":
    DATASETS_DIR = './datasets_trade/'
    OUTPUTS_DIR = "./outputs_trade/"
    variable = "trade_value"
    code = "HS6"
    location = 'location'
    
    # Read the database created in create_trade_database.ipynb
    data = pd.read_parquet(DATASETS_DIR + "df_trade_complete.parquet",engine='fastparquet')
    data = data.loc[data.location != "USA"] # Since we are adding the US states, we should remove the country to avoid duplication 
    data = data.groupby(['location', 'HS6'], as_index=False)['trade_value'].sum() # Group by location and HS6, summing the trade value
    df_temp = data.groupby(["location"]).sum() # We are adding some quality restrictions: we'll delete all countries with less than $1b trade
    remove_countries = df_temp.loc[df_temp["trade_value"]<1000000000].index 
    data = data.loc[~data.location.isin(remove_countries)]
    data = data.loc[data[variable]>0]
    print(data.shape)
    print(data.head())

elif data_choice == "labor":
    if labor_aggregation == "state":
        DATASETS_DIR = './datasets_labor_states/'
        OUTPUTS_DIR = "./outputs_labor_states/"
        variable = "EMP"
        location = "STATE_NAME"
        code = "NAICS2017"

        data= pd.read_parquet(DATASETS_DIR + "df_labor_usa_states.parquet",engine='fastparquet')
        data['NAICS2017'] = data['NAICS2017'].astype(str) # Ensure NAICS is string*
        data['naics_level'] = data['NAICS2017'].str.len() # Add a column for NAICS code length
        data = data[data.naics_level == 5]
        df_temp = data.groupby([location]).sum() 
        remove_states = df_temp.loc[df_temp[variable]<100000].index 
        data = data.loc[~data[location].isin(remove_states)]
        data = data.loc[~data[location].isin(["Puerto Rico"])]
        data = data.loc[data[variable]>0]
        print(data.shape)
        print(data.head())
        
    elif labor_aggregation == "metropolitan_area":
        DATASETS_DIR = './datasets_labor_metropolitan_area/'
        OUTPUTS_DIR = "./outputs_labor_metropolitan_area/"
        variable = "EMP"
        location = "metropolitan statistical area/micropolitan statistical area"
        code = "NAICS2017"

        data= pd.read_parquet(DATASETS_DIR + "df_labor_usa_metropolitan_area.parquet",engine='fastparquet')
        data['NAICS2017'] = data['NAICS2017'].astype(str) # Ensure NAICS is string*
        data['naics_level'] = data['NAICS2017'].str.len() # Add a column for NAICS code length
        mask_5digit = data["NAICS2017"].str.len().eq(6) & data["NAICS2017"].str.isnumeric()
        data = data[mask_5digit]
        print(data.shape)
        print(data.head())

(204144, 5)
   NAICS2017  EMP STATE  \
5     115112    8  None   
10    212321   19  None   
15    221122   88  None   
20    236115   28  None   
21    236118   53  None   

   metropolitan statistical area/micropolitan statistical area  naics_level  
5                                               10100                     6  
10                                              10100                     6  
15                                              10100                     6  
20                                              10100                     6  
21                                              10100                     6  


In [102]:
data.loc[data.NAICS2017 == "31-33"]

,NAICS2017,EMP,STATE,metropolitan statistical area/micropolitan statistical area,naics_level


## 1. Calculate RCA (Revealed Comparative Advantage)

Compute the RCA matrix to determine whether each region (state or country) is competitively exporting a product.

In [103]:
df_base = data[[location, code, variable]].copy()
df_base.head()

,metropolitan statistical area/micropolitan statistical area,NAICS2017,EMP
5,10100,115112,8
10,10100,212321,19
15,10100,221122,88
20,10100,236115,28
21,10100,236118,53


In [104]:
# Calculate the vector of export by location/country (sum c Xcp)

df_agrup_c = df_base.groupby([location])[[variable]].sum()
df_agrup_c.head()

,EMP
metropolitan statistical area/micropolitan statistical area,
10100,12282
10140,11876
10180,56193
10220,6685
10300,16166


In [105]:
# Calculate the ratio of the export of a product in all the country's exports.

df_prov = df_base.merge(df_agrup_c, how='left', on=location)
df_prov['porc_country']=df_prov[f'{variable}_x']/df_prov[f'{variable}_y' ]
df_prov.head()

,metropolitan statistical area/micropolitan statistical area,NAICS2017,EMP_x,EMP_y,porc_country
0,10100,115112,8,12282,0.000651
1,10100,212321,19,12282,0.001547
2,10100,221122,88,12282,0.007165
3,10100,236115,28,12282,0.00228
4,10100,236118,53,12282,0.004315


In [106]:
#Calculate the vector of export by product (sum c Xcp)

df_agrup_p = df_base.groupby([code])[[variable]].sum()
df_agrup_p.head()

,EMP
NAICS2017,
113110,996
113210,281
113310,26180
114111,2180
114112,812


In [107]:
# Calculate total export

df_agrup_pc = df_agrup_p[variable].sum()
df_agrup_pc

117042872

In [108]:
# Calculate the ratio of an exported product over total export

df_agrup_p[f'{variable}_total'] = df_agrup_pc
df_agrup_p['porc_product'] = df_agrup_p[variable]/df_agrup_p[f'{variable}_total']
df_agrup_p.head()

,EMP,EMP_total,porc_product
NAICS2017,,,
113110,996,117042872,0.000009
113210,281,117042872,0.000002
113310,26180,117042872,0.000224
114111,2180,117042872,0.000019
114112,812,117042872,0.000007


In [109]:
# Unify both tables and ratios to calculate RCA.

df_RCA = df_prov.merge(df_agrup_p, how='left', on=code)

df_RCA['RCA'] = df_RCA['porc_country'] / df_RCA['porc_product']
df_RCA['M'] = np.where(df_RCA['RCA'] >= 1, 1, 0)

df_RCA.drop(columns=[variable, f'{variable}_total', f'{variable}_y'],inplace=True)
df_RCA.rename(columns={f'{variable}_x': variable}, inplace=True)
    
df_RCA.head()

,metropolitan statistical area/micropolitan statistical area,NAICS2017,EMP,porc_country,porc_product,RCA,M
0,10100,115112,8,0.000651,0.000065,10.033826,1
1,10100,212321,19,0.001547,0.000142,10.864209,1
2,10100,221122,88,0.007165,0.002579,2.777783,1
3,10100,236115,28,0.00228,0.001879,1.213546,1
4,10100,236118,53,0.004315,0.003824,1.128578,1


In [110]:
# rca by country
df_RCA.groupby(location)[['M']].sum().sort_values(by='M', ascending=False)

,M
metropolitan statistical area/micropolitan statistical area,
31080,451
16980,411
19100,369
38900,368
40140,363
...,...
45880,28
46900,27
39700,14


In [111]:
# Transform matrix to numpy
locations = df_RCA[location].unique()
products = df_RCA[code].unique()

df_RCA['loc_int'] = df_RCA[location].apply(lambda x: np.where(locations == x)[0][0])
df_RCA['prod_int'] = df_RCA[code].apply(lambda x: np.where(products == x)[0][0])
df_RCA.head()

,metropolitan statistical area/micropolitan statistical area,NAICS2017,EMP,porc_country,porc_product,RCA,M,loc_int,prod_int
0,10100,115112,8,0.000651,0.000065,10.033826,1,0,0
1,10100,212321,19,0.001547,0.000142,10.864209,1,0,1
2,10100,221122,88,0.007165,0.002579,2.777783,1,0,2
3,10100,236115,28,0.00228,0.001879,1.213546,1,0,3
4,10100,236118,53,0.004315,0.003824,1.128578,1,0,4


In [112]:
len(locations), len(products)

(925, 962)

## 2. Calculate M matrix

Binary matrix indicating whether a region is competitive (RCA > 1) in each product.

In [113]:
# Create a matrix of zeros and fill
Mpa = np.zeros((df_RCA[location].nunique(), df_RCA[code].nunique()),
               dtype=np.int64)

for row in df_RCA.loc[df_RCA.M == 1].iterrows():
    Mpa[row[1].loc_int, row[1].prod_int] = 1

In [114]:
print('Cuantos elementos tienen RCA > 1 segun df_RCA:', len(df_RCA.loc[df_RCA.M == 1]))
print('Cuantos elementos tienen RCA > 1 segun Mpa:   ', Mpa.sum())

Cuantos elementos tienen RCA > 1 segun df_RCA: 109521
Cuantos elementos tienen RCA > 1 segun Mpa:    109521


### 2.1 Calculate Diversity

Sum across products for each region to get diversity: how many products a region exports competitively.

In [115]:
diversity = Mpa.sum(axis = 1)
diversity.shape

(925,)

### 2.2 Calculate Ubiquity

Sum across regions for each product to get ubiquity: how many regions are competitive in each product.

In [116]:
ubiquity = Mpa.sum(axis = 0)
ubiquity.shape

(962,)

### 2.3 Calculate Inverse Matrix

Create the inverse normalized matrix used in ECI and PCI calculations.

In [117]:
D_inv = np.diag(1/diversity)
D_inv.shape


(925, 925)

In [118]:
U_inv = np.diag(1/ubiquity)
U_inv.shape

(962, 962)

## 3. Calculate $\tilde{M}$ matrix
Double-standardized matrix used to compute eigenvectors for complexity metrics.

In [119]:
temp_Mpa_1 = Mpa.copy().astype(float)
for i in range(len(products)):
    temp_Mpa_1[:, i] = temp_Mpa_1[:, i]/ubiquity[i]

In [120]:
temp_Mpa_1.dtype, temp_Mpa_1.shape
Sccprima = np.matmul(temp_Mpa_1, Mpa.transpose())
Sccprima.shape

(925, 925)

In [121]:
Mmonio_c = np.matmul(D_inv, Sccprima)
Mmonio_c.shape

(925, 925)

## 4. Calculate ECI and PCI

Use the eigenvectors of the standardized matrix to calculate Economic Complexity Index (ECI) and Product Complexity Index (PCI).

In [122]:
def calc_complexity(mmonio):
    autovalores, autovectores = np.linalg.eig(mmonio)
    second_idx = np.where(autovalores == -np.sort(-autovalores)[1])[0][0]
    
    # second eigenvalue is the variance: the ECI
    complexity = autovectores[:, second_idx].real
    return complexity

In [123]:
# Check that the ECIs are not reversed. If they are reversed, run next cell to change their sign. This is because the eigenvector calculation has an indeterminate direction and must be determined manually.
eci_raw = calc_complexity(Mmonio_c)
eci_norm = (eci_raw - eci_raw.mean())/eci_raw.std()
eci_norm = eci_norm

# ECI results by country
df_eci = pd.DataFrame({'location': locations, 'ECI': eci_norm})
df_eci.sort_values('ECI', ascending=False, inplace=True)
df_eci = df_eci.merge(df_RCA.groupby([location])[['M']].sum().sort_values(by='M', ascending=False).reset_index().rename(columns={location:"location"}),how='left')
df_eci[0:56]

,location,ECI,M
0,16980,3.979596,411
1,31080,3.927732,451
2,14460,3.653433,291
3,41940,3.642622,141
4,17410,3.535851,303
5,35620,3.509087,346
6,41860,3.440079,275
7,17140,3.407785,272
8,37980,3.370360,315
9,12060,3.369373,354


### 4.1 Calculate $\hat{M}$ matrix

In [124]:
temp_Mpa_2 = Mpa.copy().astype(float)
for i in range(len(locations)):
    temp_Mpa_2[i, :] = temp_Mpa_2[i, :]/diversity[i]

In [125]:
Sppprima = np.matmul(Mpa.transpose(), temp_Mpa_2)
Sppprima.shape, Sppprima.dtype

((962, 962), dtype('float64'))

In [126]:
Mmonio_p = np.matmul(U_inv, Sppprima)
Mmonio_p.shape, Mmonio_p.dtype

((962, 962), dtype('float64'))

In [127]:
pci_raw = calc_complexity(Mmonio_p)


In [128]:
if data_choice == "trade":
    df_product_codes=pd.read_excel(DATASETS_DIR + "BACI/HSCodeandDescription.xlsx", sheet_name="HS22")
    df_product_codes = df_product_codes.loc[df_product_codes["Level"]== 6]
elif data_choice == "labor":
    df_product_codes= pd.read_excel(DATASETS_DIR + "2022_NAICS_Descriptions.xlsx")
    df_product_codes["Title"] = df_product_codes["Title"].str.replace("T$", "", regex=True).str.strip()
    df_product_codes['Code'] = df_product_codes['Code'].astype(str) # Ensure NAICS is string*
    df_product_codes = df_product_codes[["Code", "Title"]]
    df_product_codes.rename(columns={"Title":"Description"}, inplace=True)

In [130]:
# Let's check now results on PCI. 
# Also check that the PCIs are not reversed. If they are reversed, run next cell to change their sign. This is because the eigenvector calculation has an indeterminate direction and must be determined manually.
pci_norm = (pci_raw - pci_raw.mean())/pci_raw.std()
pci_norm = pci_norm
print('Mean and std of ICA:', pci_norm.mean(), '+/-', pci_norm.std())

df_pci = pd.DataFrame({code: products, 'PCI': pci_norm}).merge(df_product_codes[["Code", "Description"]].drop_duplicates(subset=["Code", "Description"]), how="left", left_on=code, right_on="Code")
df_pci.sort_values(by="PCI", ascending=False)[0:20]

Mean and std of ICA: 5.908879299460293e-17 +/- 1.0


,NAICS2017,PCI,Code,Description
954,336419,2.671040,336419,Other Guided Missile and Space Vehicle Parts a...
924,332112,2.484408,332112,Nonferrous Forging
933,523210,2.476129,523210,Securities and Commodity Exchanges
899,334112,2.472856,334112,Computer Storage Device Manufacturing
901,334613,2.379510,NaN,NaN
912,325130,2.246951,325130,Synthetic Dye and Pigment Manufacturing
889,311225,2.232702,311225,Fats and Oils Refining and Blending
956,212299,2.226095,NaN,NaN
845,512120,2.187997,512120,Motion Picture and Video Distribution
953,336415,2.112272,336415,Guided Missile and Space Vehicle Propulsion Un...


In [131]:
# Prepare arrays to export

df_RCA = df_RCA.merge(df_pci[[code, "PCI"]], how="left", on = code)

RCA = np.zeros((df_RCA[location].nunique(), df_RCA[code].nunique()),
               dtype=np.float64)

for row in df_RCA.loc[df_RCA.RCA != 0.0].iterrows():
    RCA[row[1].loc_int, row[1].prod_int] = row[1].RCA

value_level = np.zeros((df_RCA[location].nunique(), df_RCA[code].nunique()),
               dtype=np.float64)

# Fill RCA where you have to
for row in df_RCA.iterrows():
    value_level[row[1].loc_int, row[1].prod_int] = row[1][variable]

In [132]:
ENGINE, COMP = "fastparquet", "snappy" 

pd.DataFrame(Mpa, index=list(locations), columns=list(products))\
  .to_parquet(OUTPUTS_DIR + f"Mpa.parquet", engine=ENGINE, compression=COMP)

pd.DataFrame({"codes": list(products)})\
  .to_parquet(OUTPUTS_DIR + f"codes.parquet", engine=ENGINE, compression=COMP, index=False)

pd.DataFrame({"location": list(locations)})\
  .to_parquet(OUTPUTS_DIR + f"locations.parquet", engine=ENGINE, compression=COMP, index=False)

pd.DataFrame(np.asarray(RCA), index=list(locations), columns=list(products))\
  .to_parquet(OUTPUTS_DIR + f"RCA.parquet", engine=ENGINE, compression=COMP)

pd.DataFrame({"eci_norm": list(eci_norm)})\
  .to_parquet(OUTPUTS_DIR + f"eci.parquet", engine=ENGINE, compression=COMP, index=False)

pd.DataFrame({"pci_norm": list(pci_norm)})\
  .to_parquet(OUTPUTS_DIR + f"pci.parquet", engine=ENGINE, compression=COMP, index=False)

pd.DataFrame(value_level, index=list(locations), columns=list(products))\
  .to_parquet(OUTPUTS_DIR + f"value_level.parquet", engine=ENGINE, compression=COMP)


## 5. Proximity
Calculate product proximity: how likely it is for two products to be co-exported.

In [133]:
p_RCAmay1 = np.zeros((len(products),))

for prod in range(len(products)):
    p_RCAmay1[prod] = Mpa[:, prod].sum()

In [134]:
# Calculamos la proximidad: la probabilidad condicional mínima de que una icaa tenga el sector i RCA>1 dado tiene el sector j con RCA>1, y viceversa.
# Miramos entonces primero la cantidad de icas en que ambos sectores tienen RCA, lo sumo y divido por la mayor ubicuidad de ambos sectores.

almost_proximity = np.zeros((len(products), len(products)))

for p1 in range(len(products)-1):
    # como es una matriz simetrica, calcula unicamente la mitad de los valores:
    for p2 in range(p1+1, len(products)):
        almost_proximity[p1, p2] = np.logical_and(Mpa[:, p1], Mpa[:, p2]).sum()/max(p_RCAmay1[p1], p_RCAmay1[p2])

In [135]:
len(products)

962

In [136]:
# chequeo que no haya valores incorrectamente imputados
for i in range(len(products)):
    if not np.isclose(almost_proximity[i, :i].sum(), 0.0):
        print('something wrong at row', i)

In [137]:
proximity = almost_proximity + almost_proximity.transpose() + np.diag(np.ones(len(products)))

In [138]:
proximity.shape

(962, 962)

## 6. Calculate Density

Measures how close a new product is to the set of products a region already exports. Helps assess diversification feasibility.

$$
d_{pa} = \dfrac{\sum_{a'} M_{pa'} \Phi_{a,a'}}{\sum_{a'} \Phi_{a,a'}}
$$


In [139]:
#numerador
density_pa = np.matmul(Mpa, proximity)
print(density_pa.shape)

#denominador
for i in range(len(products)):
    density_pa[:, i] = density_pa[:, i]/proximity[:, i].sum()

(925, 962)


In [140]:
relative_density = np.zeros(density_pa.shape)

for p in range(density_pa.shape[0]):
    mask = Mpa[p] - 1
    # Acá me quedo con todos los productos donde no tengo VCR
    den_option_set = density_pa[p][mask.astype(bool)]  
    relative_density[p] = (density_pa[p] - den_option_set.mean()) / den_option_set.std()

## 7. Calculate Strategic Value (COG)

Strategic Value (COG) estimates the potential benefit of moving into a new product, considering its complexity and distance to current capabilities.

$$COG_{pa} = [\sum_{a'} \dfrac{\phi_{a,a'}}{\sum_{a''} \phi_{a'',a'}}(1 - M_{pa'}) ICA_{a'}] - (1 - d_{pa})ICA_{a}$$

In [141]:
non_RCA_all = np.ones((len(locations), len(products)), dtype=np.int64) - Mpa
non_RCA_all = non_RCA_all.astype(float)

distance_pa = np.matmul(non_RCA_all, proximity)

for i in range(len(products)):
    distance_pa[:, i] = distance_pa[:, i]/proximity[i, :].sum()

inv_distance_pa = np.ones((len(locations), len(products)), float) - distance_pa

In [142]:
ica_U01 = pci_norm - min(pci_norm)
ica_U01 = ica_U01 / max(ica_U01)

In [143]:
unif_A = np.zeros((len(locations), len(products)), float)
unif_B = np.zeros((len(locations), len(products)), float)
sum_prox = proximity.sum(axis = 0)

for l in range(len(locations)):
    if l % 13 == 0:
        print('.', end=' ')
        
    for a in range(len(products)):
        unif_A[l, a] = (proximity[a, :]*non_RCA_all[l, :]*ica_U01/sum_prox).sum()
        unif_B[l, a] = inv_distance_pa[l, a]*ica_U01[a]

unif_cog2_pa = unif_A - unif_B

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [144]:
relative_cog = np.zeros((len(locations), len(products)), float)
for idx_prov in range(len(locations)):
    relative_cog[idx_prov,] = unif_cog2_pa[idx_prov] - unif_cog2_pa[idx_prov][(1 - Mpa[idx_prov]).astype(bool)].mean()
    relative_cog[idx_prov,] /= unif_cog2_pa[idx_prov][(1 - Mpa[idx_prov]).astype(bool)].std()

In [145]:
pd.DataFrame(proximity, index=list(products), columns=list(products))\
  .to_parquet(OUTPUTS_DIR + f"proximity.parquet", engine=ENGINE, compression=COMP)

pd.DataFrame(relative_density, index=list(locations), columns=list(products))\
  .to_parquet(OUTPUTS_DIR + f"relative_density.parquet", engine=ENGINE, compression=COMP)

pd.DataFrame(relative_cog, index=list(locations), columns=list(products))\
  .to_parquet(OUTPUTS_DIR + f"relative_cog.parquet", engine=ENGINE, compression=COMP)